In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import numpy as np
import scipy.sparse as sp
import pandas as pd

In [2]:
DATASET_DIR = "C:/Users/db/Desktop/기업-수강ncf/NCF 졸업생 데이터"
train= pd.read_csv(DATASET_DIR+ "/NCF 졸업생 데이터 (1부).csv", sep='|')[['sex','ccd','bzc_cd','grup_cd']]
test = pd.read_csv(DATASET_DIR+ "/NCF 졸업생 데이터 (2부).csv", sep='|')[['sex','ccd','bzc_cd','grup_cd']]

C:\Users\db\AppData\Local\Temp\ipykernel_36440\1374857115.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  train= pd.read_csv(DATASET_DIR+ "/NCF 졸업생 데이터 (1부).csv", sep='|')[['sex','ccd','bzc_cd','grup_cd']]
C:\Users\db\AppData\Local\Temp\ipykernel_36440\1374857115.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv(DATASET_DIR+ "/NCF 졸업생 데이터 (2부).csv", sep='|')[['sex','ccd','bzc_cd','grup_cd']]


In [3]:
rating_list = np.concatenate([train.values, test.values])
n_rating_list = []
for a in rating_list:
    sex,ccd,bzc_cd,grup_cd = a
    n_rating_list.append([sex,ccd,bzc_cd,grup_cd])
rating_list = np.array(n_rating_list)

In [4]:
from sklearn.model_selection import train_test_split
# 모든 유저와 모든 item을 담음
unq_ccd, unq_bzccd,unq_grup_cd = [np.unique(rating_list[:, n+1]) for n in range(3)]
# user와 item의 개수 넣음
num_ccd, num_bzc, num_grup = len(unq_ccd), len(unq_bzccd),len(unq_grup_cd)
print('num_ccd:{} num_bzc:{} num_item:{}'.format(num_ccd, num_bzc, num_grup))
train_list = np.array(train)
test_list = np.array(test)
ccd2idx, bzcidx ,grupidx = {}, {},{}
for i, j in enumerate(unq_ccd):
     ccd2idx[j] = i
for i, j in enumerate(unq_bzccd):
     bzcidx[j] = i
for i, j in enumerate(unq_grup_cd):
     grupidx[j] = i    
train_list = [[int(li[0])-1,ccd2idx[str(li[1])], bzcidx[str(li[2])],grupidx[str(li[3])]] for li in rating_list]
# x_train, x_test = train_test_split(train_list, test_size=0.3, shuffle=True, random_state=34)
train_list = pd.DataFrame(train_list, columns = ['sex', 'ccd','bzc','item'])
x_train, x_test = train_test_split(train_list, test_size=0.3, shuffle=True, stratify =train_list[['sex', 'ccd','bzc']], random_state=34)

num_ccd:314 num_bzc:925 num_item:16695


In [5]:
class DfData(torch.utils.data.Dataset):
    def __init__(self,features,num_sex,num_ccd,num_bzc,num_grup,num_ng=0,train_mat=None,is_training=None):
        super(DfData,self).__init__()
        # self.features_ps = torch.Tensor(features).int()
        self.features_ps = features
        self.num_sex = num_sex
        self.num_bzc = num_bzc
        self.num_ccd = num_ccd
        self.num_grup = num_grup
        self.num_ng = num_ng
        self.labels = [0] * len(features)
        self.train_mat = train_mat
        self.is_training = is_training
        
        # negative sampling 
    def set_ng_sample(self):
        assert self.is_training, "no need to sampling when testing"
         
        self.features_ng = []
        for x in self.features_ps:
            # sex
            for _ in range(self.num_ng):
                s = np.random.randint(self.num_sex)
                u = np.random.randint(self.num_ccd)
                q = np.random.randint(self.num_bzc)
                j = np.random.randint(self.num_grup)
                # train set에 있는 경우 다시 뽑기
                self.features_ng.append([s,u,q,j])

        labels_ps = [1] * len(self.features_ps)
        labels_ng = [0] * len(self.features_ng)
        
        self.features_fill = torch.Tensor(self.features_ps + self.features_ng).to(torch.int64)
        self.labels_fill= torch.Tensor(labels_ps + labels_ng)
        

    def __len__(self):
        return (self.num_ng+1) * len(self.labels)
        
    def __getitem__(self,idx):
        features = self.features_fill if self.is_training else torch.Tensor(self.features_ps).to(torch.int64)
        labels = self.labels_fill if self.is_training else torch.Tensor(self.labels)
        
        user = features[idx]
        label = labels[idx]
        return user, label

In [6]:
num_sex = 2
x_train = x_train.values.tolist()
x_test = x_test.values.tolist()
train_dataset = DfData(x_train, num_sex,num_ccd,num_bzc, num_grup, num_ng=1, is_training=True)
train_loader = data.DataLoader(train_dataset, batch_size = 128, shuffle=True, num_workers=0)

In [7]:
train_dataset.set_ng_sample()
for a,b in train_loader:
    print(a)
    print(b)
    break

tensor([[    1,    44,   155,   699],
        [    0,    45,   588,  4019],
        [    0,   153,   801,  3285],
        [    0,   165,   490, 13138],
        [    1,   180,   165, 12919],
        [    0,    66,   814,  7268],
        [    0,    28,   284,  1378],
        [    0,    82,   648,    10],
        [    1,   150,   374,  6667],
        [    0,   198,   315,   328],
        [    0,    36,   755,  1236],
        [    1,   280,   712, 15219],
        [    1,   220,   852,   308],
        [    1,    14,   690, 16053],
        [    0,   228,   572,  1236],
        [    0,    57,    25,  5510],
        [    1,     1,   613,   338],
        [    0,   125,    13,  1131],
        [    0,    61,   389,  4289],
        [    0,   157,    43,  2119],
        [    0,   106,   481,    26],
        [    1,    51,   527,  5167],
        [    0,   273,   770,  3695],
        [    1,     9,   137,  8584],
        [    0,   213,   731,  1252],
        [    0,     9,   613,   434],
        [   

In [7]:
class FeaturesEmbedding(torch.nn.Module):

    def __init__(self, field_dims, embed_dim):
        super().__init__()
        self.embedding = torch.nn.Embedding(sum(field_dims), embed_dim)
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int64())
        torch.nn.init.xavier_uniform_(self.embedding.weight.data)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return self.embedding(x)

In [8]:
class FeaturesLinear(torch.nn.Module):

    def __init__(self, field_dims, output_dim=1):
        super().__init__()
        self.fc = torch.nn.Embedding(sum(field_dims), output_dim)
        self.bias = torch.nn.Parameter(torch.zeros((output_dim,)))
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int64())
    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return torch.sum(self.fc(x), dim=1) + self.bias


In [9]:
class MultiLayerPerceptron(torch.nn.Module):

    def __init__(self, input_dim, embed_dims, dropout, output_layer=True):
        super().__init__()
        layers = list()
        for embed_dim in embed_dims:
            layers.append(torch.nn.Linear(input_dim, embed_dim))
            layers.append(torch.nn.BatchNorm1d(embed_dim))
            layers.append(torch.nn.ReLU())
            layers.append(torch.nn.Dropout(p=dropout))
            input_dim = embed_dim
        if output_layer:
            layers.append(torch.nn.Linear(input_dim, 1))
        self.mlp = torch.nn.Sequential(*layers)

    def forward(self, x):
        """
        :param x: Float tensor of size ``(batch_size, embed_dim)``
        """
        return self.mlp(x)


In [10]:
class FactorizationMachine(torch.nn.Module):

    def __init__(self, reduce_sum=True):
        super().__init__()
        self.reduce_sum = reduce_sum

    def forward(self, x):
        """
        :param x: Float tensor of size ``(batch_size, num_fields, embed_dim)``
        """
        square_of_sum = torch.sum(x, dim=1) ** 2
        sum_of_square = torch.sum(x ** 2, dim=1)
        ix = square_of_sum - sum_of_square
        if self.reduce_sum:
            ix = torch.sum(ix, dim=1, keepdim=True)
        return 0.5 * ix


In [11]:
class CrossNetwork(torch.nn.Module):
    def __init__(self, input_dim, cross_layers):
        super(CrossNetwork, self).__init__()
        self.cross_layers = cross_layers
        self.cross_weights = torch.nn.ModuleList(
            [torch.nn.Linear(input_dim, 1) for _ in range(cross_layers)])

    def forward(self, x):
        x_0 = x
        x_cross = x
        for i in range(self.cross_layers):
            x_cross = x_0 * x_cross @ self.cross_weights[i].weight.t() + self.cross_weights[i].bias
        return x_cross

In [12]:
import torch
class DeepFactorizationMachineModel(torch.nn.Module):
    """
    A pytorch implementation of DeepFM.

    Reference:
        H Guo, et al. DeepFM: A Factorization-Machine based Neural Network for CTR Prediction, 2017.
    """

    def __init__(self, field_dims, embed_dim, mlp_dims,cross_layers, dropout):
        super().__init__()
        self.linear = FeaturesLinear(field_dims)
        self.fm = FactorizationMachine(reduce_sum=True)
        self.embedding = FeaturesEmbedding(field_dims, embed_dim)
        self.embed_output_dim = len(field_dims) * embed_dim
        self.mlp = MultiLayerPerceptron(self.embed_output_dim, mlp_dims, dropout)
        self.cross = CrossNetwork(self.embed_output_dim, cross_layers)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        embed_x = self.embedding(x)
        x = self.linear(x) + self.fm(embed_x) + self.mlp(embed_x.view(-1, self.embed_output_dim)) + self.cross(embed_x.view(-1, self.embed_output_dim))
        # return torch.sigmoid(x.squeeze(1))
        return x.squeeze(1)

In [13]:
sex = 2 

In [14]:
field_dims = [sex, num_ccd,num_bzc,num_grup]
embed_dim= 50
cross_layers = 1
mlp_dims = [50, 50] 
dropout= 0.2
device = 'cuda:0'

In [15]:
net = DeepFactorizationMachineModel(field_dims, embed_dim , mlp_dims, cross_layers, dropout)
net.to(device)

DeepFactorizationMachineModel(
  (linear): FeaturesLinear(
    (fc): Embedding(17936, 1)
  )
  (fm): FactorizationMachine()
  (embedding): FeaturesEmbedding(
    (embedding): Embedding(17936, 50)
  )
  (mlp): MultiLayerPerceptron(
    (mlp): Sequential(
      (0): Linear(in_features=200, out_features=50, bias=True)
      (1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.2, inplace=False)
      (4): Linear(in_features=50, out_features=50, bias=True)
      (5): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): Dropout(p=0.2, inplace=False)
      (8): Linear(in_features=50, out_features=1, bias=True)
    )
  )
  (cross): CrossNetwork(
    (cross_weights): ModuleList(
      (0): Linear(in_features=200, out_features=1, bias=True)
    )
  )
)

In [16]:
learning_rate = 0.005
epochs = 50
loss_function = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [18]:
for ep in range(epochs):   
    avg_cost = 0
    train_loader.dataset.set_ng_sample()
    for user, label in train_loader:
        user = user.to(torch.int64).to(device)
        label = label.to('cuda:0')
        net.zero_grad()
        pred = net(user)
        loss = loss_function(pred, label)
        loss.backward()
        optimizer.step()
        avg_cost += loss / len(train_loader)
    net.eval()
    # HR = metrics(net, test_loader, 10)    
    # print(np.mean(HR))
    print('Epoch:', '%04d' % (ep + 1), 'cost =', '{:.9f}'.format(avg_cost)) 

Epoch: 0001 cost = 0.140909985
Epoch: 0002 cost = 0.039061595
Epoch: 0003 cost = 0.026132993
Epoch: 0004 cost = 0.022084229
Epoch: 0005 cost = 0.021067077
Epoch: 0006 cost = 0.019773917
Epoch: 0007 cost = 0.019906111
Epoch: 0008 cost = 0.019099928
Epoch: 0009 cost = 0.018561093
Epoch: 0010 cost = 0.018969154
Epoch: 0011 cost = 0.018377854
Epoch: 0012 cost = 0.018061195
Epoch: 0013 cost = 0.017917056
Epoch: 0014 cost = 0.019029960
Epoch: 0015 cost = 0.018591674
Epoch: 0016 cost = 0.018066378
Epoch: 0017 cost = 0.018924104
Epoch: 0018 cost = 0.018629055
Epoch: 0019 cost = 0.017441109
Epoch: 0020 cost = 0.018406250
Epoch: 0021 cost = 0.017888952
Epoch: 0022 cost = 0.017600726
Epoch: 0023 cost = 0.018199245
Epoch: 0024 cost = 0.018267829
Epoch: 0025 cost = 0.017810838
Epoch: 0026 cost = 0.018111806
Epoch: 0027 cost = 0.018131357
Epoch: 0028 cost = 0.017948495
Epoch: 0029 cost = 0.018352468
Epoch: 0030 cost = 0.018370332
Epoch: 0031 cost = 0.019007785
Epoch: 0032 cost = 0.018264351
Epoch: 0

In [17]:
# torch.save(net,"C:/Users/db/Desktop/deep_fm/Deep_fm.pth")
model_path = "C:/Users/db/Desktop/deep_fm/xDeep_fm.pth"
net = torch.load(model_path)

In [30]:
# torch.save(net,"C:/Users/db/Desktop/deep_fm/xDeep_fm.pth")


In [ ]:
# 테스트셋 
# 1 . test셋에 있는 sex , ccd, grup_cd를 돌리고 -> 원래라면 실제 학습에 들어간 grup은 빼고/ 왜나면 기업은 이전에 들은것이 무엇인지 모름 /테스트 
# -> 사실상 hit - ndcg로 가는게 맞지 않나? hit - ndcg만 해가자 
# 2 . 그리고 top-k 개로 뽑아서 실제 test-set에 있는 거랑 비교 

# ncf 
#1. 각 기업에 대한 과목의 예측을 뽑고 top-k개로 그 예측이 test셋에 있는지 확인  ->

In [46]:
# device = 'cuda:0'
topn=10
user_sex=1
user_id = 8
user_ccd = 224
a = []
for i in range(num_grup):
    item = i
    a.append([user_sex,user_ccd,user_id,item])
a = torch.tensor(a).to(device)
output = net(a)
output.to(torch.int)[0:100]
# with torch.no_grad():  
#     values , sorted_indices = torch.sort(net(torch.tensor(a).to(device)), descending=True)
    
#     # 순위를 얻기 위해 정렬된 순서의 순서를 다시 정렬합니다.
#     rank = torch.argsort(sorted_indices)
#     top_10_rank = rank[:topn]
#     top_10_values = values[:topn]
    
#     # 결과 출력
#     print("상위 {}개의 순위 (rank):".format(topn), top_10_rank)
#     print("상위 {}개의 확률:".format(topn), top_10_values)

tensor([  0, -10,  -5,  -4,  -1,   3,   0, -24,   7,   0,   3,  -7, -29,  -2,
        -27,  -8,   0, -16,  -5, -17, -21, -20,  -4,  -7,  -1, -12,  -1, -22,
        -21,  -1, -12,  -7,  -1,  -2,  -8,  -6,  -5,  -9,  -2,   0,  -4,  -8,
        -13,  -4, -20, -16, -37,  -4,  -9,  -2,  -3,  -1,  -3,   7,  -5, -13,
         -6, -22, -35, -29, -38, -19, -26, -13,  -1, -17, -12, -11, -13, -15,
        -27, -28, -16, -16, -19, -25, -30, -24, -33, -20, -11, -25,  -8, -13,
        -20, -27, -22, -17, -16, -51, -46, -46, -13, -25, -23, -25, -32, -32,
        -28, -26], device='cuda:0', dtype=torch.int32)

In [ ]:
num_ccd, num_bzc, num_grup

In [83]:
x_test

[[2, 91, 750, 310],
 [2, 91, 750, 422],
 [2, 91, 750, 407],
 [2, 91, 750, 31],
 [2, 91, 750, 10],
 [2, 91, 750, 6],
 [2, 91, 750, 387],
 [2, 91, 750, 410],
 [2, 91, 750, 402],
 [2, 91, 750, 314],
 [2, 91, 750, 240],
 [2, 91, 750, 35],
 [2, 91, 750, 9],
 [2, 89, 810, 0],
 [2, 89, 810, 444],
 [2, 89, 810, 387],
 [2, 89, 810, 10],
 [2, 89, 810, 329],
 [2, 89, 810, 16],
 [2, 89, 810, 2278],
 [2, 89, 810, 402],
 [2, 89, 810, 390],
 [2, 89, 810, 364],
 [2, 89, 810, 332],
 [2, 89, 810, 313],
 [2, 89, 810, 385],
 [1, 88, 812, 410],
 [1, 88, 812, 5],
 [1, 88, 812, 2278],
 [1, 88, 812, 4],
 [1, 88, 812, 15],
 [1, 88, 812, 402],
 [1, 88, 812, 53],
 [1, 88, 812, 6],
 [1, 88, 812, 387],
 [1, 88, 812, 386],
 [1, 88, 812, 305],
 [1, 88, 812, 444],
 [2, 100, 715, 328],
 [2, 100, 715, 23],
 [2, 100, 715, 5],
 [2, 100, 715, 337],
 [2, 100, 715, 402],
 [2, 100, 715, 425],
 [2, 100, 715, 7869],
 [2, 100, 715, 7871],
 [2, 100, 715, 386],
 [2, 100, 715, 9],
 [2, 100, 715, 240],
 [2, 100, 715, 16587],
 [2, 1

In [18]:
x_train = np.array(x_train)
x_test = np.array(x_test)

In [19]:
train_dict = {}
for item in x_train:
    key = tuple(item[:3])  # 2, 91, 750을 튜플로 만듭니다.
    value = item[3]  # 나머지 값은 딕셔너리의 값으로 사용합니다.
    
    if key in train_dict:
        train_dict[key].append(value)
    else:
        train_dict[key] = [value]

In [20]:
result_dict = {}
for item in x_test:
    key = tuple(item[:3])  # 2, 91, 750을 튜플로 만듭니다.
    value = item[3]  # 나머지 값은 딕셔너리의 값으로 사용합니다.
    
    if key in result_dict:
        result_dict[key].append(value)
    else:
        result_dict[key] = [value]

In [21]:
keys_list = list(result_dict.keys())
for a in keys_list:
    print(result_dict[a])
    break

[618, 6994, 12453, 509, 14047, 8935, 8942, 8956, 1732, 368, 8987, 8947, 642, 7004, 8963, 8940, 8987, 1076, 11162, 1408, 575, 8953, 8963, 8987, 6998, 8942, 1743, 8966, 1033, 8951, 8946, 1023, 8965, 7004, 8958, 575, 8939, 1016, 1076, 1019, 373, 8961, 8987, 8221, 8987, 380, 6994, 8951, 368, 367, 1668, 374, 8987, 1008, 373, 368, 8949, 8224, 8205, 8934, 7004, 8964, 8966, 1668, 998]


In [22]:
from tqdm import tqdm  # Import tqdm for the progress bar

def precision_and_map(k_values):
    precision_results = {k: [] for k in k_values}
    map_results = {k: [] for k in k_values}
    recall_results = {k: [] for k in k_values}
    
    for i in tqdm(keys_list, desc="Calculating Precision", position=0, leave=True):
        p = []
        try:
            b = result_dict[i]
            c = train_dict[i]
        except KeyError:
            continue
        
        for j in range(num_grup):
            p.append(list(i) + [j])  # Fix variable name from 'a' to 'i'
        
        p = torch.tensor(p).to(device)
        pred = net(p).to(torch.int)  # Change to 'torch.int' for data type
        values, sorted_indices = torch.sort(pred, descending=True)
        
        original_indices = [idx for idx in sorted_indices.tolist() if idx not in set(c)]
        
        for k in k_values:
            exclusive_indices = original_indices[:k]  # Use the original indices here
            # Precision 계산
            intersection_count_k = len(set(b).intersection(set(exclusive_indices)))
            precision_results[k].append(intersection_count_k / k)
            # recall 계산 
            recall_results[k].append(intersection_count_k / len(b))
            # MAP 계산
            precision_at_i = []
            
            for rank, idx in enumerate(exclusive_indices, start=1):
                if idx in set(b):
                    precision_at_i.append(sum([1 if r in set(b) else 0 for r in exclusive_indices[:rank]]) / rank)
            
            if precision_at_i:
                map_results[k].append(sum(precision_at_i) / len(precision_at_i))
            else:
                map_results[k].append(0.0)
    return precision_results, map_results, recall_results

k_values = [10, 25, 50, 100]
precision_results, map_results, recall_results = precision_and_map(k_values)

# Access the results
precision_result_10 = precision_results[10]
precision_result_25 = precision_results[25]
precision_result_50 = precision_results[50]
precision_result_100 = precision_results[100]

map_result_10 = map_results[10]
map_result_25 = map_results[25]
map_result_50 = map_results[50]
map_result_100 = map_results[100]

recall_result_10 = recall_results[10]
recall_result_25 = recall_results[25]
recall_result_50 = recall_results[50]
recall_result_100 = recall_results[100]


Calculating Precision: 100%|███████████████████████████████████████████████████████| 9600/9600 [12:15<00:00, 13.06it/s]


In [23]:
precision_10_mean = np.mean(precision_result_10)
precision_25_mean = np.mean(precision_result_25)
precision_50_mean = np.mean(precision_result_50)
precision_100_mean = np.mean(precision_result_100)

recall_10_mean = np.mean(recall_result_10)
recall_25_mean = np.mean(recall_result_25)
recall_50_mean = np.mean(recall_result_50)
recall_100_mean = np.mean(recall_result_100)

map_10_mean = np.mean(map_result_10)
map_25_mean = np.mean(map_result_25)
map_50_mean = np.mean(map_result_50)
map_100_mean = np.mean(map_result_100)

# 평균 출력
print("Precision@10 Mean:", precision_10_mean)
print("Precision@25 Mean:", precision_25_mean)
print("Precision@50 Mean:", precision_50_mean)
print("Precision@100 Mean:", precision_100_mean)

print("Recall@10 Mean:", recall_10_mean)
print("Recall@25 Mean:", recall_25_mean)
print("Recall@50 Mean:", recall_50_mean)
print("Recall@100 Mean:", recall_100_mean)

print("MAP@10 Mean:", map_10_mean)
print("MAP@25 Mean:", map_25_mean)
print("MAP@50 Mean:", map_50_mean)
print("MAP@100 Mean:", map_100_mean)

Precision@10 Mean: 0.18488541666666666
Precision@25 Mean: 0.1467166666666667
Precision@50 Mean: 0.11504166666666668
Precision@100 Mean: 0.08438958333333334
Recall@10 Mean: 0.09999097819272576
Recall@25 Mean: 0.19769707530985323
Recall@50 Mean: 0.3077995819361884
Recall@100 Mean: 0.44838159766161007
MAP@10 Mean: 0.3486109190956475
MAP@25 Mean: 0.2990134906078117
MAP@50 Mean: 0.2533854720277903
MAP@100 Mean: 0.21371601340008298


In [22]:
from tqdm import tqdm
import torch

def calculate_ap(gt, pred, k):
    """
    단일 클래스에 대한 평균 정밀도 (AP)를 계산합니다.
    """
    precision_at_k = 0.0
    true_positives = 0

    for i in range(k):
        if pred[i] in gt:
            true_positives += 1
            precision_at_k += true_positives / (i + 1)

    if not gt:
        return 0.0

    return precision_at_k / len(gt)

def mean_average_precision(k):
    result = []

    for i in tqdm(keys_list, desc="mAP 계산 중", position=0, leave=True):
        try:
            b = result_dict[i]
            c = train_dict[i]
        except KeyError:
            continue

        p = []
        for j in range(num_grup):
            p.append(list(i) + [j])
        p = torch.tensor(p).to(device)
        pred = net(p).to(torch.int)
        values, sorted_indices = torch.sort(pred, descending=True)

        exclusive_indices = [idx for idx in sorted_indices.tolist() if idx not in set(c)] 
        exclusive_indices = exclusive_indices[:k]

        ap = calculate_ap(b, exclusive_indices, k)
        result.append(ap)

    mean_ap = sum(result) / len(result) if result else 0.0
    return mean_ap


In [23]:
map_10 = mean_average_precision(10)
map_25 = mean_average_precision(25)
map_50 = mean_average_precision(50)
map_100 = mean_average_precision(100)

mAP 계산 중: 100%|█████████████████████████████████████████████████████████████████| 9600/9600 [10:08<00:00, 15.78it/s]


In [24]:
print(map_10)
print(map_25)
print(map_50)
print(map_100)

0.05175221105177012
0.0752485485405752
0.09267475182604294
0.10793466183742993


In [165]:
# def precision(k)
#     result = []
#     for i in keys_list:  # Iterate only the first 5 elements of keys_list
#         p = []
#         try:
#             b = result_dict[i]
#             c = train_dict[i]
#         except KeyError:
#             continue
#         for j in range(num_grup):
#             p.append(list(i) + [j])  # Fix variable name from 'a' to 'i'
        
#         p = torch.tensor(p).to(device)
#         pred = net(p).to(torch.int)  # Change to 'torch.int' for data type
#         values, sorted_indices = torch.sort(pred, descending=True)
        
#         exclusive_indices = [idx for idx in sorted_indices.tolist() if idx not in set(c)]
#         exclusive_indices = exclusive_indices[:k]
        
#         intersection_count = len(set(b).intersection(set(exclusive_indices)))
#         result.append(intersection_count / k)
    
#     return result 

[0.0, 0.0, 0.7, 0.0, 0.2, 0.2, 0.3, 0.2, 0.1, 0.1]


In [24]:
from tqdm import tqdm  # Import tqdm for the progress bar

def precision(k):
    result = []
    for i in tqdm(keys_list, desc="Calculating Precision", position=0, leave=True):  # tqdm setup
        p = []
        try:
            b = result_dict[i]
            c = train_dict[i]
        except KeyError:
            continue
        for j in range(num_grup):
            p.append(list(i) + [j])  # Fix variable name from 'a' to 'i'
        
        p = torch.tensor(p).to(device)
        pred = net(p).to(torch.int)  # Change to 'torch.int' for data type
        values, sorted_indices = torch.sort(pred, descending=True)
        
        exclusive_indices = [idx for idx in sorted_indices.tolist() if idx not in set(c)]
        exclusive_indices = exclusive_indices[:k]
        
        intersection_count = len(set(b).intersection(set(exclusive_indices)))
        result.append(intersection_count / k)
    
    return result

In [25]:
# Example usage:
a_10 = precision(10)
a_25 = precision(25)
a_50 = precision(50)
a_100 = precision(100)

Calculating Precision: 100%|███████████████████████████████████████████████████████| 9600/9600 [11:57<00:00, 13.38it/s]


In [26]:
print(np.array(a_10).mean())
print(np.array(a_25).mean())
print(np.array(a_50).mean())
print(np.array(a_100).mean())

0.18488541666666666
0.1467166666666667
0.11504166666666668
0.08438958333333334


In [27]:
from tqdm import tqdm  # Import tqdm for the progress bar

def Recall(k):
    result = []
    for i in tqdm(keys_list, desc="Calculating Precision", position=0, leave=True):  # tqdm setup
        p = []
        try:
            b = result_dict[i]
            c = train_dict[i]
        except KeyError:
            continue
        for j in range(num_grup):
            p.append(list(i) + [j])  # Fix variable name from 'a' to 'i'
        
        p = torch.tensor(p).to(device)
        pred = net(p).to(torch.int)  # Change to 'torch.int' for data type
        values, sorted_indices = torch.sort(pred, descending=True)
        
        exclusive_indices = [idx for idx in sorted_indices.tolist() if idx not in set(c)]
        exclusive_indices = exclusive_indices[:k]
        
        intersection_count = len(set(b).intersection(set(exclusive_indices)))
        result.append(intersection_count / len(set(b)))
    
    return result

In [28]:
b_10 = Recall(10)
b_25 = Recall(25)
b_50 = Recall(50)
b_100 = Recall(100)

Calculating Precision: 100%|███████████████████████████████████████████████████████| 9600/9600 [06:53<00:00, 23.22it/s]


In [29]:
print(np.array(b_10).mean())
print(np.array(b_25).mean())
print(np.array(b_50).mean())
print(np.array(b_100).mean())

0.1034699039873153
0.20444689216100628
0.3183467383520758
0.46379674706775015
